In [33]:
!pip install gdown -qq

In [34]:
!pip install facenet-pytorch torchvision

In [35]:
!gdown 'https://drive.google.com/uc?export=download&id=1Mrx0OKnBFteOw1q8IZy-n8x9q8cxZwhT'
!gdown 'https://drive.google.com/uc?export=download&id=1EgvzTNEWTXvegURlmJAt8OXOtrKAlQEb'
!gdown 'https://drive.google.com/uc?export=download&id=1RcLasSJj-XMke5Fj33adiaHWbbl-9ihW'

Downloading...
From (uriginal): https://drive.google.com/uc?export=download&id=1Mrx0OKnBFteOw1q8IZy-n8x9q8cxZwhT
From (redirected): https://drive.google.com/uc?export=download&id=1Mrx0OKnBFteOw1q8IZy-n8x9q8cxZwhT&confirm=t&uuid=7aaa63ce-b4ea-46ae-86a2-7bdbdff1d475
To: /kaggle/working/post-processed.zip
100%|███████████████████████████████████████| 71.4M/71.4M [00:00<00:00, 129MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1EgvzTNEWTXvegURlmJAt8OXOtrKAlQEb
To: /kaggle/working/marcelinho_no_db.jpg
100%|██████████████████████████████████████| 37.5k/37.5k [00:00<00:00, 74.8MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1RcLasSJj-XMke5Fj33adiaHWbbl-9ihW
To: /kaggle/working/marcelinho_na_inferencia.jpg
100%|██████████████████████████████████████| 87.6k/87.6k [00:00<00:00, 93.2MB/s]


In [36]:
%%bash
rm -rf inferencia
rm -rf post-processed
mkdir inferencia
unzip -q post-processed.zip
mv *.jpg inferencia
rm post-processed.zip

### Treinando Modelo

In [37]:
import os
import cv2
import random
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms

# Carregue as imagens e crie os rótulos com base nos nomes das pastas
root_dir = "post-processed"
folders = [os.path.join(root_dir, folder) for folder in os.listdir(root_dir)]
images = []
labels = []

def process_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (116, 116))
    return img

for folder in folders:
    current_label = os.path.basename(folder)
    image_paths = glob(os.path.join(folder, "*.jpg"))
    for image_path in image_paths:
        image = process_image(image_path)
        images.append(image)
        labels.append(current_label)

images = np.array(images)
labels = np.array(labels)

# Organize as imagens carregadas e rótulos em um dicionário
data_dict = {}
for image, label in zip(images, labels):
    if label not in data_dict:
        data_dict[label] = []
    data_dict[label].append(image)
    
# CustomTripletDataset
class CustomTripletDataset(Dataset):
    def __init__(self, data_dict):
        self.data_dict = data_dict
        self.labels = list(data_dict.keys())
        self.transform = transforms.ToTensor()

    def __len__(self):
        return sum([len(v) for v in self.data_dict.values()])

    def __getitem__(self, _):
        # Função para verificar se uma lista possui pelo menos dois itens distintos
        def has_unique_elements(items):
            items_set = []
            for item in items:
                if any((item == x).all() for x in items_set):
                    continue
                items_set.append(item)
                if len(items_set) > 1:
                    return True
            return False

        # Filtrando os rótulos que possuem pelo menos duas imagens distintas
        valid_labels = [label for label in self.labels if has_unique_elements(self.data_dict[label])]

        anchor_label = random.choice(valid_labels)
        negative_label = random.choice([l for l in valid_labels if l != anchor_label])

        anchor = random.choice(self.data_dict[anchor_label])
        positive = random.choice([img for img in self.data_dict[anchor_label] if not np.array_equal(img, anchor)])
        negative = random.choice(self.data_dict[negative_label])

        anchor, positive, negative = self.transform(anchor), self.transform(positive), self.transform(negative)

        return anchor, positive, negative

# FaceNet
class FaceNet(nn.Module):
    def __init__(self):
        super(FaceNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(64, 192, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm2d(192)
        self.pool2 = nn.MaxPool2d(3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(192, 384, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(384, 256, kernel_size=5, padding=2)
        self.pool4 = nn.MaxPool2d(3, stride=2, padding=1)
        self.fc1 = nn.Linear(8*8*256, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 128)

    def forward(self, x):
        x = self.pool1(self.bn1(F.relu(self.conv1(x))))
        x = self.pool2(self.bn2(F.relu(self.conv2(x))))
        x = F.relu(self.conv3(x))
        x = self.pool4(F.relu(self.conv4(x)))
        x = x.view(-1, 8*8*256)
        x = F.dropout(F.relu(self.fc1(x)), training=self.training)
        x = F.dropout(F.relu(self.fc2(x)), training=self.training)
        x = self.fc3(x)

        return x

# Instancie o CustomTripletDataset e divida o conjunto de dados em treino e validação
dataset = CustomTripletDataset(data_dict)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_data, val_data = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_data, batch_size=32, shuffle=True, num_workers=2)

# Verifique se a GPU está disponível
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Instancie o modelo e envie-o para a GPU
model = FaceNet().to(device)

# Defina o otimizador e a função de perda (Perda Tripla)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.TripletMarginLoss(margin=1.0)

def train(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0

    for anchor, positive, negative in tqdm(loader, desc="Training"):
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)
        optimizer.zero_grad()

        anchor_output = model(anchor)
        positive_output = model(positive)
        negative_output = model(negative)

        loss = criterion(anchor_output, positive_output, negative_output)
        running_loss += loss.item()

        loss.backward()
        optimizer.step()

    return running_loss / len(loader)

def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0

    with torch.no_grad():
        for anchor, positive, negative in tqdm(loader, desc="Validation"):
            anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

            anchor_output = model(anchor)
            positive_output = model(positive)
            negative_output = model(negative)

            loss = criterion(anchor_output, positive_output, negative_output)
            running_loss += loss.item()

    return running_loss / len(loader)

In [38]:
# Treinar a rede por várias épocas com Early Stopping
num_epochs = 10
patience = 2
counter = 0
best_loss = None

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    val_loss = validate(model, val_loader, criterion, device)
    print("Epoch: {}/{} - Training Loss: {:.4f} - Validation Loss: {:.4f}".format(epoch + 1, num_epochs, train_loss, val_loss))

    if best_loss is None or val_loss < best_loss:
        best_loss = val_loss
        counter = 0
    else:
        counter += 1
        print(f"EarlyStopping counter: {counter}/{patience}")
        if counter >= patience:
            print("EarlyStopping: Stopping training")
            break

Training:   0%|          | 0/300 [00:00<?, ?it/s]

Validation:   0%|          | 0/75 [00:00<?, ?it/s]

Epoch: 1/10 - Training Loss: 1.0514 - Validation Loss: 1.0000


Training:   0%|          | 0/300 [00:00<?, ?it/s]

Validation:   0%|          | 0/75 [00:00<?, ?it/s]

...

Validation:   0%|          | 0/75 [00:00<?, ?it/s]

Epoch: 3/10 - Training Loss: 0.9980 - Validation Loss: 1.0000
EarlyStopping counter: 1/2


Training:   0%|          | 0/300 [00:00<?, ?it/s]

Validation:   0%|          | 0/75 [00:00<?, ?it/s]

Epoch: 4/10 - Training Loss: 1.0071 - Validation Loss: 1.0000
EarlyStopping counter: 2/2
EarlyStopping: Stopping training


In [40]:
model_path = "FaceNet_model.pth"

# Salvar o modelo (estado do dicionário)
torch.save(model.state_dict(), model_path)

In [41]:
!ls

FaceNet_model.pth  __notebook_source__.ipynb  inferencia  post-processed
